In [1]:
import os
import glob
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
config = ({"input_size": 1024,
           "image_size": 512,
           "backbone": "efficientnetb7",
           "method": "unet",
           "class": "outline"
          })

In [17]:
CLASSES = ['building', 'outline']

In [18]:
x_valid_dir = './building/val/raw/'
y_valid_dir = './building/val/outline_label/'

In [19]:
# valid dataset raw image 불러오기
x_valid_list = glob.glob(x_valid_dir + '*.png')

In [20]:
x_valid_list = sorted(x_valid_list)

In [21]:
import albumentations as A

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        A.Lambda(image=preprocessing_fn),
        A.Resize(config['image_size'], config['image_size'])
    ]
    return A.Compose(_transform)

In [22]:
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    x *= 255
    return x

In [23]:
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras


In [24]:
# 전처리
BACKBONE = config['backbone']
preprocess_input = sm.get_preprocessing(BACKBONE)
preprocessing = get_preprocessing(preprocess_input)

In [25]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

In [26]:
# 불러올 모델 파일 위치
model.load_weights('/home/ssac10/aiffel/hackerton3/wandb/run-20210610_133952-3f0ge1hb/files/model-best.h5')

In [27]:
from tqdm import tqdm

In [28]:
# 리스트에서 이미지 불러서 저장하는 함수
for file in tqdm(x_valid_list):
    filename = os.path.basename(file)
    # 저장 경로 : ./images/class_backbone_image_size
    savepath = f'./images/{config["class"]}/{config["backbone"]}_{config["image_size"]}'
    if not os.path.exists(savepath):
        os.mkdir(savepath)
    image = cv2.imread(file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    preprocessed = preprocessing(image=image)
    image = preprocessed['image']
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    output = denormalize(np.squeeze(pr_mask, axis=0))
    output = cv2.resize(output, (config['input_size'], config['input_size']))
    cv2.imwrite(os.path.join(savepath, filename), output)

100%|██████████| 159/159 [00:49<00:00,  3.19it/s]
